In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import deque

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # Detects cars, trucks (excluding motorcycles)
helmet_model = YOLO('helmet-detector.pt')  # Detects drivers, helmets, and no-helmets

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30  # Default FPS if unreadable
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
previous_positions = {}
previous_frames = {}
object_ids = {}  # Assign unique IDs
next_id = 1  # Counter for new objects

# Calibration factor (adjust based on real-world measurement)
PIXEL_TO_METER = 0.0046  

# Speed tracking
speed_buffer = {}
MAX_SPEED = 150  # km/h
max_speed_dict = {}

# Define excluded classes
EXCLUDED_CLASSES = {3}  # COCO class ID for motorcycles

# Processing video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles (excluding motorcycles)
    vehicle_results = vehicle_model.track(frame, persist=True)

    current_positions = {}

    # Process vehicle detections
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            if box.id is None:
                continue  

            class_id = int(box.cls[0])  
            if class_id in EXCLUDED_CLASSES:
                continue  # Skip motorcycles

            obj_id = int(box.id[0])  
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            # Assign unique ID
            if obj_id not in object_ids:
                object_ids[obj_id] = next_id
                next_id += 1  

            unique_id = object_ids[obj_id]
            current_positions[obj_id] = center

            # Speed Calculation
            if obj_id in previous_positions:
                prev_center = previous_positions[obj_id]
                prev_frame = previous_frames[obj_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = 1 / fps  

                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            speed_kmph = min(speed_kmph, MAX_SPEED)  

            # Store max speed
            if unique_id not in max_speed_dict:
                max_speed_dict[unique_id] = speed_kmph
            else:
                max_speed_dict[unique_id] = max(max_speed_dict[unique_id], speed_kmph)

            # Smooth speed calculation
            if unique_id not in speed_buffer:
                speed_buffer[unique_id] = deque(maxlen=5)
            speed_buffer[unique_id].append(speed_kmph)
            smoothed_speed = np.mean(speed_buffer[unique_id])

            # Draw bounding box & speed
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {unique_id}", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame, f"Speed: {int(smoothed_speed)} km/h", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            cv2.putText(frame, f"Max Speed: {int(max_speed_dict[unique_id])} km/h", (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

            # Update tracking info
            previous_positions[obj_id] = center
            previous_frames[obj_id] = cap.get(cv2.CAP_PROP_POS_FRAMES)

    # Detect drivers & helmets
    helmet_results = helmet_model.predict(frame, stream=True)

    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            # Filter out "Unknown" detections
            if class_id not in [1, 2, 3]:  # Only allow 'Driver', 'Helmet', 'No Helmet'
                continue

            color = (255, 255, 255)  
            label = ""

            if class_id == 1:  # Driver
                color = (0, 255, 255)  
                label = "Driver"
            elif class_id == 2:  # Helmet
                color = (0, 255, 0)  
                label = "Helmet"
            elif class_id == 3:  # No Helmet
                color = (0, 0, 255)  
                label = "No Helmet"

            # Draw driver & helmet detections
            cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 2)
            cv2.putText(frame, label, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save processed frame
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Processing complete. Video saved at: {output_video_path}")


100%|██████████| 21.5M/21.5M [05:44<00:00, 65.6kB/s]



0: 384x640 1 bicycle, 5 cars, 1 suitcase, 134.8ms
Speed: 2.3ms preprocess, 134.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 99.2ms
Speed: 2.5ms preprocess, 99.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 103.2ms
Speed: 3.0ms preprocess, 103.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 97.2ms
Speed: 1.0ms preprocess, 97.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 100.0ms
Speed: 1.0ms preprocess, 100.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 97.2ms
Speed: 2.0ms preprocess, 97.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 6 cars, 98.0ms
Speed: 2.0ms preprocess, 98.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no det

In [5]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # Detects cars, trucks (Excludes motorcycles)
helmet_model = YOLO('helmet-detector.pt')  # Detects drivers, helmets, no-helmets

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'  # Replace with actual video path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output1.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}  # Maps YOLO object IDs to unique tracking IDs
next_id = 1  # Counter for assigning unique IDs

# Define excluded classes
EXCLUDED_CLASSES = {3}  # Class ID for motorcycles (COCO dataset)

# Processing video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles (excluding motorcycles)
    vehicle_results = vehicle_model.track(frame, persist=True)

    # Process vehicle detections
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            if box.id is None:
                continue  

            class_id = int(box.cls[0])  
            if class_id in EXCLUDED_CLASSES:
                continue  # Skip motorcycles

            obj_id = int(box.id[0])  
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Assign unique ID
            if obj_id not in object_ids:
                object_ids[obj_id] = next_id
                next_id += 1  

            unique_id = object_ids[obj_id]

            # Draw bounding box (thinner lines)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)  
            cv2.putText(frame, f"ID: {unique_id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Detect drivers & helmets
    helmet_results = helmet_model.predict(frame, stream=True)

    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id not in [1, 2, 3]:  # Only allow 'Driver', 'Helmet', 'No Helmet'
                continue

            color = (255, 255, 255)  
            label = ""

            if class_id == 1:  # Driver
                color = (0, 255, 255)  
                label = "Driver"
            elif class_id == 2:  # Helmet
                color = (0, 255, 0)  
                label = "Helmet"
            elif class_id == 3:  # No Helmet
                color = (0, 0, 255)  
                label = "No Helmet"

            # Draw driver & helmet detections with thinner boxes
            cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)
            cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Save processed frame
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 1 bicycle, 5 cars, 1 suitcase, 119.5ms
Speed: 3.5ms preprocess, 119.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 129.2ms
Speed: 2.0ms preprocess, 129.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 137.5ms
Speed: 2.0ms preprocess, 137.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 135.3ms
Speed: 2.0ms preprocess, 135.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 137.4ms
Speed: 2.1ms preprocess, 137.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 151.0ms
Speed: 1.0ms preprocess, 151.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 6 cars, 141.3ms
Speed: 3.2ms preprocess, 141.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640

In [12]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import deque

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # Detects cars, trucks (Excludes motorcycles)
helmet_model = YOLO('helmet-detector.pt')  # Detects drivers, helmets, no-helmets

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/highway.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}  # Maps YOLO object IDs to unique tracking IDs
next_id = 1  # Counter for assigning unique IDs
previous_positions = {}  # Store previous frame positions
previous_frames = {}  # Store previous frame numbers
speed_buffer = {}  # Store speed history for smoothing
max_speed_dict = {}  # Store the **actual max speed** detected per object

# Calibration factor (adjust based on real-world measurement)
PIXEL_TO_METER = 0.05  

# Define excluded classes
EXCLUDED_CLASSES = {3}  # Exclude motorcycles from vehicle detection

while cap.isOpened():
    ret, frame = cap.read()
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    if not ret:
        break

    # Detect vehicles (excluding motorcycles)
    vehicle_results = vehicle_model.track(frame, persist=True)

    # Process vehicle detections
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            if box.id is None:
                continue  

            class_id = int(box.cls[0])  
            if class_id in EXCLUDED_CLASSES:
                continue  # Skip motorcycles

            obj_id = int(box.id[0])  
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            # Assign unique ID
            if obj_id not in object_ids:
                object_ids[obj_id] = next_id
                next_id += 1  

            unique_id = object_ids[obj_id]

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]

                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps

                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            # Store **actual max speed** detected
            if unique_id not in max_speed_dict:
                max_speed_dict[unique_id] = speed_kmph
            else:
                max_speed_dict[unique_id] = max(max_speed_dict[unique_id], speed_kmph)

            # Save tracking data
            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            # Draw bounding box (thin lines)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)  
            cv2.putText(frame, f"ID: {unique_id} {int(max_speed_dict[unique_id])} km/h", (x1, y1 - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Detect drivers & helmets
    helmet_results = helmet_model.predict(frame, stream=True)

    driver_boxes = []
    helmet_boxes = []

    # Separate driver and helmet detections
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                driver_boxes.append((hx1, hy1, hx2, hy2))
            elif class_id in [2, 3]:  # Helmet or No Helmet
                helmet_boxes.append((hx1, hy1, hx2, hy2, class_id))

    # Process drivers
    for hx1, hy1, hx2, hy2 in driver_boxes:
        obj_id = len(object_ids) + 1  # Assign unique ID to driver
        object_ids[obj_id] = obj_id
        unique_id = obj_id

        # Speed Calculation for Drivers
        center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

        if unique_id in previous_positions:
            prev_center = previous_positions[unique_id]
            prev_frame = previous_frames[unique_id]

            pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
            time_elapsed = (current_frame - prev_frame) / fps

            speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
        else:
            speed_kmph = 0

        # Store **actual max speed** detected
        if unique_id not in max_speed_dict:
            max_speed_dict[unique_id] = speed_kmph
        else:
            max_speed_dict[unique_id] = max(max_speed_dict[unique_id], speed_kmph)

        # Draw bounding box and max speed for driver
        cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for driver
        cv2.putText(frame, f"ID: {unique_id} {int(max_speed_dict[unique_id])} km/h", (hx1, hy1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Process helmets & associate with nearest driver
    for hx1, hy1, hx2, hy2, class_id in helmet_boxes:
        color = (0, 255, 0) if class_id == 2 else (0, 0, 255)  # Green for helmet, Red for no-helmet
        label = "Helmet" if class_id == 2 else "No Helmet"

        # Find nearest driver
        nearest_driver = min(driver_boxes, key=lambda d: abs(d[0] - hx1) + abs(d[1] - hy1), default=None)
        if nearest_driver:
            driver_x1, driver_y1, _, _ = nearest_driver
            cv2.putText(frame, label, (driver_x1, driver_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Save processed frame
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 9 cars, 3 trucks, 143.6ms
Speed: 4.7ms preprocess, 143.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 helmet, 143.1ms
Speed: 2.3ms preprocess, 143.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 201.7ms
Speed: 4.0ms preprocess, 201.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 helmet, 181.2ms
Speed: 4.0ms preprocess, 181.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 190.2ms
Speed: 11.0ms preprocess, 190.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 helmet, 223.2ms
Speed: 3.1ms preprocess, 223.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 232.3ms
Speed: 3.0ms preprocess, 232.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 helmet, 140.0ms
Speed: 3.1ms preprocess, 140.0ms inferenc

KeyboardInterrupt: 